In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize

# Start-Up Information

In [ ]:
HWP_angs = np.array([0., 11.25, 22.5, 33.75, 45., 56.25, 67.5, 78.75, 90, 
                     101.25, 112.5, 123.75, 135., 146.25, 157.5, 168.75])
IMR_angs = np.array([45., 57.5, 70., 82.5, 95., 107.5, 120., 132.5])
wavelengths = np.array([625, 675, 725, 750, 775])
angles = [HWP_angs, IMR_angs]